# Pyspark Project On Airline Data :

In [2]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 43.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=bfe787fd6ac96b0305e99c23072865d2e64bc938b6cf7513f6fd0e85e005ed97
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [3]:
## importing required libraries

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Assignment').getOrCreate()
import pyspark.sql.functions as f

In [5]:
## reading the dataset through pyspark 

airline = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/content/Airline_data.csv")
airline.show()

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|1989|    1|        23|        1|   1419|      1230|   1742|      1552|           UA|      183

### Q1. Show a sample of 5 records from dataset.

In [6]:
## by using 'show' function printing first 5 observations from dataset

airline.show(5)

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|1989|    1|        23|        1|   1419|      1230|   1742|      1552|           UA|      183

### Q2. Read the data with data types.

In [7]:
## Using "printSchema" function to get the data type of all the attributes 

airline.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: integer (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: integer (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: integer (nullable = true)
 |-- CRSElapsedTime: integer (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: integer (nullable = true)
 |-- DepDelay: integer (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |

### Q3. Make a new column MonthStr, Which has months in form of 01, 02, 03, ..., 12.

In [13]:
## importing user defined function and col from pyspark sql

from pyspark.sql.functions import udf, col

In [19]:
## defining the user defined function by passing the colunm month as parameter 

def checkvalue(Month):
  if len(str(Month)) == 1:
    return ('0' + str(Month))
  else :
    return str(Month)

a = udf(checkvalue)

### creating the new column as monthstr and converting the month numbers in form of 01,02 .... 12

airline = airline.withColumn("MonthStr", a(col("Month")))
airline.show()

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+--------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|MonthStr|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+--------+
|1989|    1|        23|        1|   1419|      1230|   1742|      1

### Q4. Find the # of flights each airline made.


In [10]:
## using group by function to not repeat the values 
## and finding the count of flights each airline made  (that UA airline made)

airline.groupBy("UniqueCarrier").count().show()

+-------------+-----+
|UniqueCarrier|count|
+-------------+-----+
|           UA|  426|
+-------------+-----+



### Q5. Find the mean Arrival delay per origination airport.

In [11]:
## using groupy by function to Origign to get in the form of unique values
## and finding the mean of arrival delay per origination airport
 
airline.groupby("Origin").mean("ArrDelay").show()

+------+-------------------+
|Origin|      avg(ArrDelay)|
+------+-------------------+
|   LIH|0.16666666666666666|
|   HNL|  14.21774193548387|
|   EWR|               9.25|
|   DEN| 20.166666666666668|
|   IAD| 12.966666666666667|
|   SFO| 11.215384615384615|
|   PHL|  6.827586206896552|
|   OGG|  16.24137931034483|
+------+-------------------+



### Q6. What is the average departure delay from each airport?

In [12]:
## Using group by function to origin 
## And finding the average of departure delay from each airport

airline.groupby("Origin").avg("DepDelay").show()

+------+-------------------+
|Origin|      avg(DepDelay)|
+------+-------------------+
|   LIH|-3.7666666666666666|
|   HNL|  3.217741935483871|
|   EWR|  4.958333333333333|
|   DEN|               27.6|
|   IAD|                8.9|
|   SFO| 19.646153846153847|
|   PHL| 16.137931034482758|
|   OGG|                6.0|
+------+-------------------+

